In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import splitfolders

tfd = tfp.distributions
tfpl = tfp.layers

In [2]:
train_dir = os.path.join('C:/Users/Mfund/Downloads/COVID-radiography/COVID-19_Radiography_Dataset_splitted/train/')
test_dir = os.path.join('C:/Users/Mfund/Downloads/COVID-radiography/COVID-19_Radiography_Dataset_splitted/val/')

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def image_generator(train_par_dir, test_par_dir):
    train_datagen = ImageDataGenerator(rescale= 1/255)
    test_datagen = ImageDataGenerator(rescale=1/255)
    
    train_generator= train_datagen.flow_from_directory(train_par_dir,
                                                     target_size = (75,75),
                                                     batch_size = 256,
                                                     class_mode = 'categorical',
                                                     subset = 'training')
    test_generator = test_datagen.flow_from_directory(test_par_dir,
                                                     target_size = (75,75),
                                                     batch_size = 256,
                                                     class_mode = 'categorical')
    
    return train_generator, test_generator

In [10]:
train_generator,test_generator = image_generator(train_dir,test_dir)

Found 16930 images belonging to 4 classes.
Found 4235 images belonging to 4 classes.


In [49]:
tf.keras.backend.clear_session()

kl_divergence_function = lambda q, p, _: tfd.kl_divergence(q, p) / tf.cast(16930, dtype=tf.float32)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(126,126,1),name="basket"),
    
    tfp.layers.Convolution2DFlipout(16, kernel_size=16, strides=(1,1), data_format="channels_last", 
                                    padding="same", activation=tf.nn.relu, name="conv_tfp_1a", 
                                    kernel_divergence_fn=kl_divergence_function),
    tf.keras.layers.MaxPool2D(strides=(4,4), pool_size=(4,4), padding="same"),
    
    tfp.layers.Convolution2DFlipout(32, kernel_size=3, strides=(1,1), data_format="channels_last", 
                                    padding="same", activation=tf.nn.relu, name="conv_tfp_1b", 
                                    kernel_divergence_fn=kl_divergence_function),
    tf.keras.layers.MaxPool2D(strides=(4,4), pool_size=(4,4), padding="same"),

    tf.keras.layers.Flatten(),
    tfp.layers.DenseFlipout(1, kernel_divergence_fn=kl_divergence_function),
])



In [50]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_tfp_1a (Conv2DFlipout)  (None, 126, 126, 16)      8208      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 16)        0         
_________________________________________________________________
conv_tfp_1b (Conv2DFlipout)  (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense_flipout (DenseFlipout) (None, 1)                 4097      
Total params: 21,553
Trainable params: 21,553
Non-trainable params: 0
____________________________________________________

In [51]:
from tensorflow.keras.optimizers import Adam

def negative_log_likelihood(y_true, y_pred):
    return -y_pred.log_prob(y_true)

model.compile(loss = negative_log_likelihood,
              optimizer = Adam(learning_rate=0.5),
              metrics = ['accuracy'],
              experimental_run_tf_function = False)

In [53]:
learning_rate = 1.0e-3
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate),
              metrics=['mse'])

In [54]:
labels = os.listdir(test_dir)
batch_size = 32
history = model.fit(train_generator, 
                    
                    epochs=10, 
                    verbose=1, 
                    batch_size=batch_size,
                    validation_freq=5)

Epoch 1/10


InvalidArgumentError:  output depth must be evenly divisible by number of groups: 16 vs 3
	 [[node sequential/conv_tfp_1a/Conv2D (defined at C:\Users\Mfund\anaconda3\lib\site-packages\tensorflow_probability\python\layers\conv_variational.py:1076) ]] [Op:__inference_train_function_16504]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/conv_tfp_1a/Conv2D:
 IteratorGetNext (defined at <ipython-input-54-bd3a757be4f9>:3)

Function call stack:
train_function


In [52]:
history_bayes = model.fit(
      train_generator,
      epochs=10,
      verbose=1)

Epoch 1/10


AttributeError: in user code:

    C:\Users\Mfund\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:850 train_function  *
        return step_function(self, iterator)
    <ipython-input-47-142a41d20756>:4 negative_log_likelihood  *
        return -y_pred.log_prob(y_true)
    C:\Users\Mfund\anaconda3\lib\site-packages\tensorflow\python\framework\ops.py:401 __getattr__
        self.__getattribute__(name)

    AttributeError: 'Tensor' object has no attribute 'log_prob'
